<a href="https://colab.research.google.com/github/taskswithcode/GPTTokenizationTutorial/blob/main/GPTTokenizationTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notebook to examine GPT's byte level BPE tokenization process**

In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 83.1 MB/s eta 0:00:00


In [3]:
from transformers import GPT2Tokenizer

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [5]:
resp= tokenizer("a")['input_ids']

In [6]:
resp

[64]

In [7]:
resp= tokenizer("அ ஆ இ ஈ உ ")['input_ids']

In [8]:
resp

[156,
 106,
 227,
 220,
 156,
 106,
 228,
 220,
 156,
 106,
 229,
 220,
 156,
 106,
 230,
 220,
 156,
 106,
 231,
 220]

In [9]:
resp= tokenizer("அ")['input_ids'] #The UTF 8 encoding for அ is 0xE0 AE 85

In [10]:
resp

[156, 106, 227]

[156,106,227] is the indices of the symbols representing those bytes in the vocab.json file. In the vocab.json file, index 156 is the character  "à"; 106 is the character ® and 227 is the character ħ. 

The UTF-8 encoding for à is 0x C3 A0 in UTF-8. Expanding this 1100 0011 1010 0000. Removing the control bits and aggregating the rest we get E0. This is indeed the first byte of the UTF-8 encoding of அ

Similarly the UTF-8 encoding for à is 0x C2 AE in UTF-8. Expanding this 1100 0010 1010 1110. Removing the control bits and aggregating the rest we get AE. This is indeed the second byte of the UTF-8 encoding of அ

The UTF-8 encoding for the third byte,  ħ is 0x C4 A7. Expanding this 1100 0100 1010 0111. Removing control bits and aggregating we get 0x127. This does not match the third byte of அ which is 0x85. This is simply because 0x85 is a control character in ISO-8859-1 the standard for 1 byte representations. These are mapped to visible unicode characters. In this case the mapping is to ħ

In [ ]:
def bytes_to_unicode():
    bs = ( 
        list(range(ord("!"), ord("~") + 1)) 
        + list(range(ord("¡"), ord("¬") + 1)) 
        + list(range(ord("®"), ord("ÿ") + 1)) 
    )   
    cs = bs[:]
    n = 0 
    missing = []
    missing_map = []
    for b in range(2 ** 8): 
        if b not in bs: 
            bs.append(b)
            cs.append(2 ** 8 + n)
            missing.append(b)
            missing_map.append(2 ** 8 + n)
            n += 1
    cs = [chr(n) for n in cs] 
    return dict(zip(bs, cs)),dict(zip(missing,missing_map))

In [ ]:
#Displays all the mapped characters for the all 256 bit patterns in a byte. This mapping maps all control characters to visible unicode characters/
#Note the model only sees the mapped indices during training.
x,missing_dict = bytes_to_unicode()
count = 0 
for key in x:
    count += 1
    print(f"{count}] {key} {key:02x} {x[key]}")

In [ ]:
#To see all the control character mappings only
count = 0 
for key in missing_dict:
    count += 1
    print(f"{count}] {key} {key:02x} {x[key]} {missing_dict[key]}")